In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/27 12:22:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data = spark.read.parquet('./data/vuelos')

In [4]:
data.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [5]:
data.show()

25/04/27 12:26:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [6]:
from pyspark.sql.functions import col

In [7]:
nuevo_df = (data.filter(col('MONTH').isin(6, 7, 8))
            .withColumn('dis_tiempo_aire', col('DISTANCE') / col('AIR_TIME'))
            ).select(
                col('AIRLINE'),
                col('dis_tiempo_aire')
            ).where(col('AIRLINE').isin('AA', 'DL', 'AS'))

In [8]:
nuevo_df.explain(True)

== Parsed Logical Plan ==
'Filter 'AIRLINE IN (AA,DL,AS)
+- Project [AIRLINE#66, dis_tiempo_aire#250]
   +- Project [YEAR#62, MONTH#63, DAY#64, DAY_OF_WEEK#65, AIRLINE#66, FLIGHT_NUMBER#67, TAIL_NUMBER#68, ORIGIN_AIRPORT#69, DESTINATION_AIRPORT#70, SCHEDULED_DEPARTURE#71, DEPARTURE_TIME#72, DEPARTURE_DELAY#73, TAXI_OUT#74, WHEELS_OFF#75, SCHEDULED_TIME#76, ELAPSED_TIME#77, AIR_TIME#78, DISTANCE#79, WHEELS_ON#80, TAXI_IN#81, SCHEDULED_ARRIVAL#82, ARRIVAL_TIME#83, ARRIVAL_DELAY#84, DIVERTED#85, ... 8 more fields]
      +- Filter MONTH#63 IN (6,7,8)
         +- Relation [YEAR#62,MONTH#63,DAY#64,DAY_OF_WEEK#65,AIRLINE#66,FLIGHT_NUMBER#67,TAIL_NUMBER#68,ORIGIN_AIRPORT#69,DESTINATION_AIRPORT#70,SCHEDULED_DEPARTURE#71,DEPARTURE_TIME#72,DEPARTURE_DELAY#73,TAXI_OUT#74,WHEELS_OFF#75,SCHEDULED_TIME#76,ELAPSED_TIME#77,AIR_TIME#78,DISTANCE#79,WHEELS_ON#80,TAXI_IN#81,SCHEDULED_ARRIVAL#82,ARRIVAL_TIME#83,ARRIVAL_DELAY#84,DIVERTED#85,... 7 more fields] parquet

== Analyzed Logical Plan ==
AIRLINE: str